In [ ]:
import actr
import pickle
import random
import os
import fnmatch
import sys
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import groupby
import json
import re

### Observed Issues:
(see file "current_issues")

- Recency effect still too weak.

- Primacy effect too strong.



#### Number of words in each list i.e. list_length should be n where (n-2)%3 == 0 because 2 neutral words are added in each list.

#### The adjustable parameters in this experiment code.
    - number of agents (top of next cell)
    - experimental conditions (further below) --> multiple conditions can be set in a list of lists [[rehearsal time1, num words per list 1], [rehearsal time 2, num words per list 2], ...]
    - Number of lists (further below)
#### Adjustable parameters in ACT-R
    - :declarative-num-finsts 21 ; number of items that are kept as recently retrieved (Change it to 5) 
    - :declarative-finst-span 21 ; how long items stay in the recently-retrieved state (5,100)

In [ ]:
# help(actr)

In [ ]:
### Experiment part ###

def __init__(iteration, rehearsal_time, list_length, list_amount=3, path="."):
    subject = ''

    current_list = ''
    
    associated_list = ''

    recalled_words = defaultdict(list)

    rehearsed_words =  defaultdict(lambda: defaultdict(int))

    #list_amount = 3   # No of lists (100, 200, 1000, 2000 AND 5000)

    #Set below where function is actually called
    list_length = list_length   # No of words in a list
    rehearsal_time = rehearsal_time  # No of seconds for which rehearsal happens and each word is shown

    delay = 1  #delay between rehearsal and recall

    recall_time = 90
    
    #distractor_time = 30 #set further below
    
    word_lists_dict = defaultdict(list)
    
    # Ensure there are enough unique words to create the word lists
    word_dict = {"positive": ["positive" + str(i) for i in range(999)],
                 "negative": ["negative" + str(i) for i in range(999)],
                 "neutral": ["neutral" + str(i) for i in range(999)]}

    filename = f'{path}\words_{list_length}_lists_{list_amount}_rh_time_{rehearsal_time}_rec_time_{recall_time}_delay_{delay}_{iteration}.txt'
    
    results = {}

    results['x'] = {'data': [], # will be appended later in the analytics function
                          'info': "Storing range(len(word_lists_dict[0])) here"}

    results['rehearse_frequency'] = {'data': None,# will be appended later in the analytics function
                                     'info': "Storing list(rehearse_frequency.values()) here"}

    results['recall_probability'] = {'data': None, # will be appended later in the analytics function
                                        'info': "Storing list(recall_probability.values()) here"}

    results['first_recall'] = {'data': None, # will be appended later in the analytics function
                               'info': "Storing list(first_probability.values()) here"}
    
    results['pli'] = {'data': None, # will be appended later in the analytics function
                         'info': "Storing avg PLIs per agent here"}
    
#     results['transitions'] = {'data': None, # will be appended later in the analytics function
#                          'info': "Storing recall transitions here"}
    
#     results['neg_thought_train'] = {'data': None, # will be appended later in the analytics function
#                          'info': "Storing negative thought train lengths here"}

    with open(filename, 'w') as outfile:
        json.dump(results, outfile)

    with open(filename) as json_file:
        results = json.load(json_file)
#         #print(results)
    
    globals().update(locals())  ## Making everything public, worst code you can ever write!!

In [ ]:
def add_words(i, list_length):
    '''
    Add the words to the word lists, ensures valence categories are balanced
    '''
    global word_lists_dict

    amnt_wanted = (list_length)
    amt_neutral, count = 0, 0
    while len(word_lists_dict[i]) != list_length:
        count += 1
        #print(f"...................{count,word_lists_dict[i]}")
        if count >= 9999: # IF it takes too long to create a unique list at random, start over
            word_lists_dict[i] = []
            add_words(i, list_length)
        if len(word_lists_dict[i]) < list_length: # Place two neutral words at the start to control for primacy effects
            word_to_add = word_dict["neutral"][random.randint(0, len(word_dict["neutral"])-1)]
            if word_to_add not in word_lists_dict[i]:
                word_lists_dict[i].append(word_to_add)


def create_lists(list_amount=3, list_length=2):
    '''
    Create the wordlists used during the free recall tasks 
    '''  
    global word_lists_dict 

    for i in range(list_amount):
        print(f'List {i+1}/{list_amount} created!', end="\r")
        add_words(i, list_length)

    # Save the dictionary to a .pickle file, so we do not have to create the word lists everytime we run the model                    
    file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}_neutral.pickle","wb")
    pickle.dump(word_lists_dict, file)
    file.close()
    return word_lists_dict

# Check if the word lists already exist, else create new word lists
def check_and_create_lists():
    global word_lists_dict
    try:
        file = open(f"word_lists/word_lists_dict_{list_length}_{list_amount}_neutral.pickle","rb")
        #file = open(f"word_lists_dict_100_items_only.pickle","rb")
        word_lists_dict = pickle.load(file)  
        file.close()
        print("\nSuccesfully loaded the word lists!\n")
    except FileNotFoundError:
        print("\nCreating word lists!\n")
        #amount_to_create = list_amount                              
        word_lists_dict = create_lists(list_amount,list_length)

def display_word_lists():
    '''
    Display the word lists loaded/created
    '''
    for key, value in word_lists_dict.items():
        print(f'List {key}:\n {value}\n')

def close_exp_window():
    '''
    Close opened ACT-R window
    '''
    return actr.current_connection.evaluate_single("close-exp-window")


def prepare_for_memorization():
    '''
    Enable rehearsing productions to start the memorization phase 
    '''    
    
    #actr.run(1, False)
    
    enable_list = ["rehearse-first", "rehearse-second", "rehearse-third", "rehearse-fourth", 
                    "rehearse-first-default", "rehearse-second-default", "rehearse-third-default",
                    "rehearse-fourth-default", "rehearse-it", "rehearse-it-wrong-word", "skip-rehearse-1",
                    "skip-rehearse-2", "skip-rehearse-3", "skip-rehearse-4", "attend-new-word", "attend-new-word-2",
                    "initiate-rumination", "continue-rumination-1", "continue-rumination-2",
                    "wait-1", "wait-2", "wait-3", "wait-4", "ruminate", "finish-recall-1",
                   "finish-recall-2"]
    
    disable_list = ["retrieve-a-word", "recall-a-word", "stop-recall", "ruminate"]
    
    for prod in disable_list:
        actr.pdisable(prod)
        
    for prod in enable_list:
        actr.penable(prod)   
    
    actr.goal_focus("goal") # set goal to start memorization
    
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff)  
    
    #actr.run(1, False)
    
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff)

    
def prepare_for_recall(distractor=0): 
    '''
    Disable rehearsing productions, and clearing buffer contents to start the recalling phase 
    '''
    if distractor: #disable distractor or memorization PRs
        
        disable_list = ["crowd-out-wm", "continue-task"]
    
    else:
        
        disable_list = ["rehearse-first", "rehearse-second", "rehearse-third", "rehearse-fourth", 
                        "rehearse-first-default", "rehearse-second-default", "rehearse-third-default",
                        "rehearse-fourth-default", "rehearse-it", "rehearse-it-wrong-word", "skip-rehearse-1",
                        "skip-rehearse-2", "skip-rehearse-3", "skip-rehearse-4", "attend-new-word", "attend-new-word-2",
                       "initiate-rumination", "continue-rumination-1", "continue-rumination-2",
                        "wait-1", "wait-2", "wait-3", "wait-4", "ruminate", "finish-recall-1",
                       "finish-recall-2"]
    
    #enable list is the same either way
    enable_list = ["retrieve-a-word", "recall-a-word", "stop-recall", "ruminate"]
    
    for prod in disable_list:
        actr.pdisable(prod)

    for prod in enable_list:
        actr.penable(prod)
        
    actr.run(delay, False) 
    
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff) 
        
def prepare_for_distractor(): 
    '''
    Disable rehearsing productions, and clearing buffer contents to start the distractor task 
    '''
    disable_list = ["rehearse-first", "rehearse-second", "rehearse-third", "rehearse-fourth", 
                    "rehearse-first-default", "rehearse-second-default", "rehearse-third-default",
                    "rehearse-fourth-default", "rehearse-it", "rehearse-it-wrong-word", "skip-rehearse-1",
                  "skip-rehearse-2", "skip-rehearse-3", "skip-rehearse-4", "attend-new-word", "attend-new-word-2",
                   "initiate-rumination", "continue-rumination-1", "continue-rumination-2",
                   "wait-1", "wait-2", "wait-3", "wait-4", "ruminate", "finish-recall-1",
                   "finish-recall-2"]
    
    enable_list = ["crowd-out-wm", "continue-task"]
    
    for prod in disable_list:
        actr.pdisable(prod)

    for prod in enable_list:
        actr.penable(prod)
        
    actr.run(delay, False) 
    
    for buff in ["imaginal", "retrieval", "production"]:
        actr.clear_buffer(buff) 

def setup_dm(word_lists_dict, rumination_chunks=0):
    '''
    Add words to declarative memory, since it can be assumed the test subjects know the English language already
    '''
    #print("\n\n############################################# Inside setup_dm i.e. Declarative Memory")
    
    rumination_list = []
    if rumination_chunks > 0:
        for i in range(rumination_chunks):
            actr.add_dm(('rumination'+str(i), 'isa', 'memory', 'word', 'rumination'+str(i),
                         'valence', 'RED', 'context', 'rumination')) #, 'type', 'rumination'
            rumination_list.append(('rumination'+str(i)).upper())
    
        #set creation times and times of prior reference for each chunk  
    actr.sdp(':reference-list', list(np.linspace(0, -1000, 600)), ':creation-time', -1000)
    
    
    colour_conversion = {'pos': 'GREEN', 'neg': 'RED', 'neu': 'BLACK'}
    #adding the remaining words (positive + neutral after setting connection strengths between rumination chunks and negative words)
    for list_idx, word_list in word_lists_dict.items():
        i, j = 0, 0
        for idx, word in enumerate(word_list):
            valence = ''.join([char for char in word if not char.isdigit()])[:3]
            #we have to first add only the negative words s.t. we can set the similarities with the rumination chunks via sdp (sdp would otherwise set the similarities for all chunks)
            if valence == 'pos':
                actr.add_dm((valence+str(i)+"-"+str(list_idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence],
                            'context', 'list'+str(list_idx), 'type', 'on-task')) #, 'type', 'on-task'
                i += 1
            elif valence == 'neu':
                actr.add_dm((valence+str(j)+"-"+str(list_idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence],
                            'context', 'list'+str(list_idx), 'type', 'on-task')) #, 'type', 'on-task'
                j += 1
                

                
    #         if idx == 0:
    #             print("\n Emaple of a chunk added in Declarative Memory is \n")
    #             print('item'+str(idx), 'isa', 'memory', 'word', "'"+word+"'", 'valence', colour_conversion[valence],"\n")

    
def setup_experiment(human=True):
    '''
    Load the correct ACT-R model, and create a window to display the words
    '''
#     print("\n\n############################################# Inside setup_experiment")
#     print(f'\nSubject = {subject}\n')  

    loaded = None
    if subject == "controls":
        loaded = actr.load_act_r_model(r"C:\Users\cleme\Documents\Education\RUG\First-Year_Research\My_Project\Model\models\general_free_recall_model_Murdock_Roberts.lisp")
        #loaded = actr.load_act_r_model(r"C:\Users\cleme\Documents\Education\RUG\First-Year_Research\My_Project\Model\models\csm_free_recall_model.lisp")
    elif subject == "depressed":
        loaded = actr.load_act_r_model(r"C:\Users\cleme\Documents\Education\RUG\First-Year_Research\My_Project\Model\models\rumination_free_recall_model_alt_v1_originalSolution.lisp")
    
    #print("\n\n############################################# Inside setup_experiment")
    #print(f'\nLoaded Act-r model = {loaded}\n')  

    window = actr.open_exp_window("Free Recall Experiment", width=1024, height=768, visible=human) # 15inch resolution window
    actr.install_device(window) 
    return window    

# def record_associated_list(item):
#     '''
#     Register which list the recalled words belong to (to identify prior-list intruisions)
#     '''
#     associated_list = item

def record_words_recalled(item1, item2):
    '''
    Register which words were recalled during the experiment for a specific wordlist and strip the numbers
    '''
    valence = ''.join(char for char in item1 if not char.isdigit())
    item_idx = ''.join(char for char in item1 if char.isdigit())
    context = str(item2)
    
    if str(item2) != 'RUMINATION':
        recalled_words[current_list].append((valence, item_idx, context))
    
    #print(valence, item_idx, context)

def record_words_rehearsed(item):
    '''
    Register amount of rehearsals per word for each wordlist
    '''
    rehearsed_words[current_list][item] += 1

def create_lplot(idx, xlabel, ylabel, x, y, xticks_len, filename, ytick_range=None, show=False):
    '''
    Create line plot using matplotlib
    '''
    plt.figure(idx)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(x, y)
    plt.xticks(np.arange(0, xticks_len, 1)) 
    plt.yticks(ytick_range)
    #plt.savefig("images/"+subject+"_"+filename, bbox_inches='tight')
    if show:
        plt.show()    

        
def create_result_dict():
    '''
    Use a module-level function, instead of lambda function, to enable pickling it
    '''
    return defaultdict(int)

## Creating different pickle files to store results from multiple hyper-parameter values.


def analysis(wlist_amount, show_plots=False):
    '''
    Review results of the recall experiment
    '''
    global results
    result_dict = defaultdict(create_result_dict) # instead of defaultdict(lambda: defaultdict(int))
    first_recall = defaultdict(int)
    recall_probability = defaultdict(int)
    rehearse_frequency = defaultdict(int)
    pli_dict = defaultdict(int)
    transitions_amnt = 0
    thought_train_len = []
    

#     for key, val in recalled_words.items():
#         thought_train_len.extend([(k, sum(1 for _ in count)) for k, count in groupby([val[0] for val in val[2:]])])
#         for idx, (retrieved_word, item_num, _) in enumerate(val[2:]):
#             if idx != 0:
#                 if retrieved_word != val[2:][idx-1][0]:
#                     transitions_amnt += 1/wlist_amount # average over word lists
#     print(f'Avg. Amount of recall transitions = {int(transitions_amnt)}')
    
#     neg_thought_train_len = 0
#     neg_divider = 0.0001
#     for x in thought_train_len:
#         if x[0] == 'negative':
#             neg_divider += 1
#             neg_thought_train_len += x[1]
#     avg_neg_thought_train_len = round(neg_thought_train_len/neg_divider, 3)
#     print(f'Avg. Negative Thought train length = {avg_neg_thought_train_len}')            
    
    for list_num, wlist in word_lists_dict.items():
        if list_num < wlist_amount:
            for key, val in recalled_words.items():
                if key==list_num:
                    first_recall[wlist.index(''.join(val[0][0:2]))] += 1
                    for idx, word in enumerate(wlist):
                        first_recall[idx] += 0
                        if ((''.join(char for char in word if not char.isdigit()), 
                             ''.join(char for char in word if char.isdigit()), val[0][2])) in val:
                            recall_probability[idx] += 1
                        else:
                            recall_probability[idx] += 0
#                 for retrieved_word, item_num, list_idx in val[2:4]:
#                     result_dict["pstart"][retrieved_word] += 1  
#                 for retrieved_word, item_num, list_idx in val[4:-2]:
#                     result_dict["pstay"][retrieved_word] += 1
#                 for retrieved_word, item_num, list_idx in val[-2:]:
#                     result_dict["pstop"][retrieved_word] += 1 
            for key, val in rehearsed_words.items():
                if key==list_num:
                    for idx, word in enumerate(wlist):
                        rehearse_frequency[idx] += rehearsed_words[key][word]
    
    
    for key, val in first_recall.items():
        first_recall[key] = val/wlist_amount

    for key, val in recall_probability.items():
        recall_probability[key] = val/wlist_amount

    for key, val in rehearse_frequency.items():
        rehearse_frequency[key] = val/wlist_amount      
        
    
    #Calculate prior-list intrustions
    #subjects committed an average of 0.61 PLIs per list (Zaromb et al., 2006)
    
    if len(recalled_words) > 1:
        overall_pli = 0
        for key, value in recalled_words.items():
            if key > 0:
                intruding_list = []
                pli = 0
                for word_info in value:
                    associated_list = int(re.findall(r'\d+', word_info[2])[0]) #extract just the number from the context and turn into int
                    if associated_list != key:
                        intruding_list.append(associated_list)
                        pli += 1

                print(f'PLIs on list {key}: {pli}')
                if intruding_list:
                    print(f'The PLIs came from {intruding_list}')
                overall_pli += pli
                
        
            pli_dict[key] = overall_pli
            
        avg_pli = overall_pli/(len(recalled_words)-1)
        print(f'Average number of PLIs: {avg_pli}') #minus 1 because no PLI possible on LIST0
       
    xticks_len = len(word_lists_dict[0])
    
    #results['x']['data'].append(range(len(word_lists_dict[0])))
    #results['xticks_len']['data'].append(len(word_lists_dict[0]) )
    results['rehearse_frequency']['data'] = list(rehearse_frequency.values())
    results['recall_probability']['data'] = list(recall_probability.values())
    results['first_recall']['data'] = list(first_recall.values())
    results['pli']['data'] = list(pli_dict.values())
    
    with open(filename, 'w') as outfile:
        json.dump(results, outfile)
        
    create_lplot(0, 'Serial input position', 'Rehearse Frequency', range(len(word_lists_dict[0])), list(rehearse_frequency.values()), 
                xticks_len, f'rehearse_frequency_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', None, show_plots)

    create_lplot(1, 'Serial input position', 'Starting Recall', range(len(word_lists_dict[0])), list(first_recall.values()), 
                xticks_len, f'starting_recall_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', np.arange(0, .5, .1), show_plots)                

    create_lplot(2, 'Serial input position', 'Recall Probability', range(len(word_lists_dict[0])), list(recall_probability.values()), 
                xticks_len, f'recall_probability_{list_length}_{list_amount}_{rehearsal_time}_{recall_time}_{delay}.png', np.arange(0, 1, .1), show_plots)   
    
#     create_lplot(3, 'Serial input position', 'Accuracy', range(len(word_lists_dict[0])), list(recall_accuracy.values()), 
#                 xticks_len, 'recall_accuracy.png', np.arange(0, 1, .1), show_plots) 

    file = open("results_"+subject+".pickle","wb")
    pickle.dump(result_dict, file)
    file.close()
    
    print(result_dict)

    return result_dict

def do_experiment(subj="controls", human=False, wlist_amount=2000, distractor_time=0, rumination_chunks=0):
    '''
    Run the experiment
    '''
    check_and_create_lists()
    global word_lists_dict, subject
    
    subject = subj
    
    assert wlist_amount <= len(word_lists_dict), "Chosen too many lists, choose less or create more word lists using function: create_lists()"
    
#     print("###################################################\n")
#     print("The original word list \n")
#     print(display_word_lists())
    
#     print("\n###################################################\n")
#     print("Experiment started, Trying to understand the flow\n")
    
    #buffers_to_clear = ['goal', "imaginal", "retrieval", "production"]
    
    
    actr.reset()
    
    window = setup_experiment(human)
    
    setup_dm(word_lists_dict, rumination_chunks=rumination_chunks)

    for idx, (key, value) in enumerate(word_lists_dict.items()):
        
        #window = setup_experiment(human)

        global current_list
        current_list = idx # keep track for which list words are recalled

        #setup_dm(key, value)
                
        prepare_for_memorization()
        
        actr.mod_focus('context', 'list'+str(current_list))
        
        #actr.run(2, human)
        
        #actr.load_act_r_code('~;Users;cleme;Documents;Education;RUG;First-Year_Research;My_Project;Model;modelsset_all_base_levels.lisp')
    
        actr.add_command("retrieved-word", record_words_recalled,"Retrieves recalled words.")
        actr.add_command("rehearsed-word", record_words_rehearsed,"Retrieves rehearsed words.")
        
    
#         print("\n##################  Model started rehearsal ")
        for idx, word in enumerate(value):
            actr.mod_focus('context', 'list'+str(current_list))
            if "neutral" in word:
                color = "black"
            elif "positive" in word:
                color = "green"
            else:
                color = "red"
            actr.add_text_to_exp_window(window, word, x=475-len(word) , y=374, color=color, font_size=20) # change later 
            #print(idx, word, f'list{key}')
            
            actr.run(rehearsal_time, human) # True when choosing Human, False when choosing differently
            
            #actr.whynot_dm()
            #actr.print_dm_finsts()
            #print(actr.buffer_chunk('goal'))
#             actr.buffer_status('retrieval')
#             actr.buffer_status('imaginal')
            
            actr.clear_exp_window(window)
            actr.run(0.5, human)  # 500-ms blank screen 
        
        if distractor_time:
            prepare_for_distractor()
            actr.mod_focus('state', 'begin-task')
            actr.run(distractor_time, human)
            
        
        prepare_for_recall(distractor_time)
        
        actr.goal_focus('startrecall')
        actr.mod_focus('context', 'list'+str(current_list))

        #actr.goal_focus('startrecall')
       
        #for buff in ["imaginal", "retrieval", "production"]:
        #    actr.clear_buffer(buff)
        
        
        #actr.remove_command("rehearsed-word")
        
#         print("\n##################  Model finished rehearsal, list of rehearsed words is ")
#         print(f'{rehearsed_words}\n')
#         print("\n##################  Model started recall ")
        #actr.goal_focus("startrecall") # set goal to start recalling
    
        actr.run(recall_time, human)  
        
#         for buffer in buffers_to_clear:
#             actr.clear_buffer(buffer)
    
        #actr.remove_command("retrieved-word")
        
#         print("\n##################  Model finished recall, list of recalled words is ")
#         print(f'{recalled_words}\n')
        print(f'Experiment {idx+1}/{wlist_amount} completed!', end="\r")
        if idx == wlist_amount-1: # run for a chosen amount of word lists
            break
    close_exp_window() # close window at end of experiment
    
    num_recalled, num_recalled_unique = 0, 0
    for key, val in recalled_words.items():
        correct_recalls = []
       # print(key, current_list)
       # print(val)
        for word in val:
            if word[2] == f'LIST{key}':
                correct_recalls.append(word[0:2])
                num_recalled += 1
        num_recalled_unique += len(set(correct_recalls))
       # print(f'\n\nList {key} (length={len(correct_recalls)}, unique={len(set(correct_recalls))})\n')
        
    avg_recalled = num_recalled//wlist_amount
    avg_unique_recalled = num_recalled_unique//wlist_amount
    
    print(f'Avg. Number of words recalled = {avg_recalled}')
    print(f'Avg. Number of unique words recalled = {avg_unique_recalled}')
    
    #analysis(list_amount, False)
    results = analysis(list_amount, False)        

    for key, val in results.items():
        print(f'{key} = {dict(val)}')
    print()
 

    print("\n\n#############################################")
    print(f'\n[{subject}] Results!\n')
    return avg_recalled, avg_unique_recalled, results

In [ ]:
num_agents = 500 #set number of agents per condition
num_lists = 3 #number of lists per agent
distractor_time = 0
rumination_chunks = 0
#subject = 'controls' # 'controls', 'depressed'

#experimental conditions [rehearsal time, list length] from Murdock 1962
experimental_setup = [[2,10],[2,15],[2,20],[1,20],[1,30],[1,40]]

#to save files to different directory for different numbers of agents
output_path = f'./murdock/agents_{num_agents}/'
if not os.path.isdir(output_path):
    os.mkdir(f'murdock/agents_{num_agents}/')
path = output_path

pstats = {}
results_dict = {}
valences = ['neutral', 'positive', 'negative']
for parameter in experimental_setup:
    rh = parameter[0]
    ll = parameter[1]
    
    total_recalled = 0
    total_unique = 0
    
    
    print("-------------------------------------------------------------------")
    print("Experimental condition:")
    print(f"Number of lists: {num_lists}")
    print(f"Words per list: {ll}")
    print(f"Rehearsal time: {rh} seconds")
    
    for agent in range(num_agents):
        
        print("-------------------------------------------------------------------")
        print(f"Started for agent_{agent}")
        print(f"Words per list: {ll}, rehearsal time: {rh} sec")
        __init__(agent, rehearsal_time=rh, list_length=ll, list_amount=num_lists, path=path)

        try:
            num_recalled, num_unique, results = do_experiment('controls',False,
                                                                list_amount,
                                                                distractor_time=distractor_time,
                                                                rumination_chunks=rumination_chunks)
            total_recalled += num_recalled
            total_unique += num_unique
        
        except ValueError:
            print("\nAgent recalled 0 items.")

    avg_recall = total_recalled / num_agents
    avg_unique = total_unique / num_agents
    
    
    results_dict[f'Rate: {rh}, Items: {ll}'] = [avg_recall, avg_unique]
    
    print(f"Experimental Condition\nWords per list: {ll}, rehearsal time: {rh} sec")
    print(f"\nAverage number of recalled words ({num_agents} agents): {avg_recall}")
    print(f"Unique: {avg_unique}")

for key, value in results_dict.items():
    print(f'\n{key}')
    print(f'\nAverage number of recalled words: {value[0]}')
    print(f'\nAverage number of unique words: {value[1]}')
    print('-------------------------------------------------------------\n')
        

In [ ]:
max_list_length = 40
f, ax = plt.subplots(1) #set up plot

#path = f'murdock/agents_{num_agents}_controls_vV/'
#path = f'murdock/agents_{num_agents}/'

f.set_size_inches(20, 10.5)

for key, value in results_dict.items():
    print(f'\n{key}')
    print(f'\nAverage number of recalled words: {value[0]}')
    print(f'\nAverage number of unique words: {value[1]}')
#     print(f'\nAverage length of negative thought trains: {round(value[2],2)}')
#     print(f'\nAverage number of recall transitions: {round(value[3],2)}')
    print('-------------------------------------------------------------\n')
    
for parameter in experimental_setup:
    rh = parameter[0] #rehearsal time
    ll = parameter[1] #words per list

    files = [] #Storing all the relevant files to work on them later
    pattern = f"words_{ll}_lists_{num_lists}_rh_time_{rh}*.txt" # Pattern for matching the filename for data retrieval
    for file in os.listdir(path): #Lists all the files and directories within the folder
        if fnmatch.fnmatch(file, pattern): #matches the above declared patter with the filenames from listdir()
            files.append(file) #Appends the file to make it available for later use.
    
    # Initializing all the parameters needed for the plots
    idx = [0,1,2]
    xlabel = 'Serial Input Position'
    ylabel = ['Rehearse Frequency','Starting Probability','Recall Probability']
    xticks_len = max_list_length+10
    rehearse_frequency = []
    recall_probability = []
    first_recall = []
    pli_list = []
    neg_thought_train = []
    transitions = []
    
    for file in files: #load the result files
        with open(f"{path}/{file}") as f:
            recall_stats = json.load(f)
            if recall_stats['recall_probability']['data']:
                recall_probability.append(recall_stats['recall_probability']['data'])
            else: #if an agent failed to recall anything...
                recall_probability.append([0]*ll)
            if recall_stats['pli']['data']:
                pli_list.append(recall_stats['pli']['data'])
            else:
                pli_list.append([0]*ll)
#             if results['neg_thought_train']['data']:
#                 neg_thought_train.append(results['neg_thought_train']['data'])
#             else:
#                 neg_thought_train.append([0]*ll)
#             if results['transitions']['data']:
#                 transitions.append(results['transitions']['data'])
#             else:
#                 transitions.append([0]*ll)
                
                
                
    #calculating avg stats per input position across agents            
    avg_recall_probs = [sum(x)/num_agents for x in zip(*recall_probability)]
    avg_pli = [sum(x)/num_agents for x in zip(*pli_list)]
    
    
#     avg_neg_train = [sum(x)/num_agents for x in zip(*neg_thought_train)]
#     avg_transitions = [sum(x)/num_agents for x in zip(*transitions)]
    
   # print(f'{ll}-{rh}\nP(First_Item): {round(avg_recall_probs[0],2)}\nP(Final_Item): {round(avg_recall_probs[-1],2)}\nAvg PLI: {sum(avg_pli)/(num_lists-1)}')
    #print(f'\nAverage negative thought train length: {round(sum(avg_neg_train)/(num_lists),2)}\nAvg num transitions: {round(sum(avg_transitions)/(num_lists),2)}')
    
    #plot results
    x = range(ll)
    ax.plot(x, avg_recall_probs, marker = "o", label = f"{ll} items, {rh} sec") #, color = 'red' 
    #ax.set_xlim([0,ll])
    plt.xticks(np.arange(0, max_list_length, step=5), fontsize = 18)
    plt.yticks(fontsize = 18)# Set label locations
    ax.set_ylim([0,1])
    ax.set_ylabel('Recall probability', fontsize = 20)
    ax.set_xlabel('Serial input position', fontsize = 20)
    
ax.legend()
ax.legend(title='Condition', bbox_to_anchor=(1.05, 1), loc='upper left') #fontsize='xx-small')

#("murdock/images/"+subject+"_"+filename, bbox_inches='tight')


# for condition, stats in pstats.items():
#     print(f'\n{condition}')
#     for stat, vals in stats.items():
#         print(f'\n{stat}')
#         total = sum(vals.values())
#         for val, res in vals.items():
#             print(f'{val}: {round(res/total,2)}')

In [ ]:
# rehearsal_times = [8,4,2,1,0.5]
# number_of_items = [10,20,30,40]
# num_lists = 3

# f, ax = plt.subplots(1) #set up plot
# f.set_size_inches(8, 10)
# for time in rehearsal_times:
#     avg_recalls = []
#     for num in number_of_items:
#         for key, value in results_dict.items():
#             if key == f'Rate: {time}, Items: {num}':
#                 avg_recalls.append(value[1])
    
#     ax.plot(number_of_items, avg_recalls, marker = "o", label = f"{time} seconds") #, color = 'red' 
#     #ax.set_xlim([0,ll])
#     plt.xticks(np.arange(0, max(number_of_items)+20, step=10), fontsize = 18)
#     plt.yticks(fontsize = 18)# Set label locations
#     ax.set_ylim([0,25])
#     ax.set_ylabel('Number recalled', fontsize = 20)
#     ax.set_xlabel('List length', fontsize = 20)

# ax.legend()
# #ax.legend(title='Condition', bbox_to_anchor=(1.05, 1), loc='upper left') #fontsize='xx-small')
                

##### do_experiment flow:

1. check_and_create_lists
	a. create_lists
	   return word_lists_dict
		i. add_words
            adds to word_lists_dict

2. setup_experiment
    returns window
      model is loaded


3. setup_dm


4. prepare_for_recall


5. close_exp_window


6. analysis
    return result_dict
        contains pstart, pstay, pstop
    prints Avg recall transitions
    
    prints Avg negative thought train length
   

In [ ]:
objects = []
with(open(r"word_lists\word_lists_dict_20_3.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
#for idx, (key, val) in enumerate(word_lists_dict.items()):
#    print(idx, (key, val))

In [ ]:
file = open(f"word_lists/word_lists_dict_30_3.pickle","rb")
#file = open(f"word_lists_dict_100_items_only.pickle","rb")
word_lists_dict = pickle.load(file)  
file.close()

word_lists_dict

In [ ]:
#actr.reset()
#setup_dm(word_lists_dict)
#actr.sdp(':creation-time', -1000, ':reference-list', list(np.linspace(0, -1000, 50)))


In [ ]:
#word_lists_dict

In [ ]:
# recalled_words.items()

In [ ]:
#     for key, val in recalled_words.items():
#         print(key, current_list)
#         print(val)
#         for word in val:
#             print(word[0:2])

In [ ]:
#actr.load_act_r_model(r"C:\Users\cleme\Documents\Education\RUG\First-Year_Research\My_Project\Model\models\rumination_free_recall_model_v1.lisp")

In [ ]:
#setup_dm(word_lists_dict)